In [2]:
import pygame
import random

screen_width = 400
screen_height = 400
play_width = 200  
play_height = 400  
block_size = 20
top_left_x = 0
top_left_y = screen_height - play_height

# Create a list of lists with the figures that are going to be used,and their possivel moves.  
I = [[' * ',
      ' * ',
      ' * '],
     ['***',
      '   ',
      '   ']]
R = [['***',
      '*  ',
      '*  '],
     ['***',
      '  *',
      '  *'],
     ['  *',
      '  *',
      '***'],
      ['*  ',
       '*  ',
       '***']]
O = [['***',
      '* *',
      '***',]]
N = [['***',
      '* *',
      '* *'],
     ['***',
      '  *',
      '***'],
     ['* *',
      '* *',
      '***'],
      ['***',
       '*  ',
       '***']]

figures = [I, R, O, N]
figure_colors = [(192, 192, 192), (192, 192, 192), (192, 192, 192), (192, 192, 192)]

pygame.font.init() # Initializes the font 
class Piece:
    """
    This class creates the figures' attributes for game play.  
    """
    rows = 20  # This refers to the number of rows considering the y-axis
    columns = 10  # This refers to the number of columns considering the y-axis 
    def __init__(self, column, row, figure):
        self.x = column # position in x
        self.y = row # position in y
        self.figure = figure # the figure itself
        self.color = figure_colors[figures.index(figure)] # Each color is assigned to each figure.
        self.rotation = 0  # number from 0-3 which refers to the index of the position in the figure's list.
    
def create_grid(locked_positions={}):
    """
    This function creates the game playground or grid, applying RGB code colors,
    and iterating through the range of columns and rows. 
    """
    grid = [[(0,51,102) for x in range(10)] for x in range(20)] # Coloring in the grid
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (j,i) in locked_positions:
                c = locked_positions[(j,i)]
                grid[i][j] = c
    return grid
    
def convert_figure_format(figure):
    """
    This function transforms a list of selected characters that ressemble a figure to create 
    an actual figure within the grid.
    An empty list is created to append tuples with the position of the figures.     
    In order to do this, The enumerate() method adds a counter to an iterable and returns it in a form of enumerate object. 
    This enumerate object can then be used directly in for loops or be converted into a list of tuples using list() method.
    
    """
    positions = []
    figure_format = figure.figure[figure.rotation % len(figure.figure)] 
 
    for i, line in enumerate(figure_format):
        row = list(line)
        for j, column in enumerate(row):
            if column == '*': # When iterating through each list of figures, if there is an '*', then its appended to the list
                positions.append((figure.x + j, figure.y + i))
 
    for i, pos in enumerate(positions):
        positions[i] = (pos[0] - 2, pos[1] - 4)
 
    return positions

def valid_space(figure, grid):
    accepted_positions = [[(j, i) for j in range(10) if grid[i][j] == (0,51,102)] for i in range(20)]
    accepted_positions = [j for sublist in accepted_positions for j in sublist]
    formatted_figure = convert_figure_format(figure)
 
    for pos in formatted_figure:
        if pos not in accepted_positions:
            if pos[1] > -1:
                return False
 
    return True

def clear_rows(grid, locked):
    inc = 0
    for i in range(len(grid)-1,-1,-1):
        row = grid[i]
        if (0, 51, 102) not in row:
            inc += 1
            # add positions to remove from locked
            ind = i
            for j in range(len(row)):
                try:
                    del locked[(j, i)]
                except:
                    continue
    if inc > 0:
        for key in sorted(list(locked), key=lambda x: x[1])[::-1]:
            x, y = key
            if y < ind:
                newKey = (x, y + inc)
                locked[newKey] = locked.pop(key)
    return inc

def check_lost(positions):
    for pos in positions:
        x, y = pos
        if y < 1:
            return True
    return False

def get_figure():
    global figures, figure_colors 
    return Piece(5, 0, random.choice(figures)) #This is the initial position of the figure: Column = 5, row = 0.

def draw_window(surface, grid, score=0):
    surface.fill((0, 0, 102))

    font = pygame.font.SysFont('gillsansultra', 60)
    label = font.render('IRON', 1, (0, 0, 255))
    font1 = pygame.font.SysFont('playbill', 80)
    label1 = font1.render('TETRIS', 1, (160, 160, 160))
    font2 = pygame.font.SysFont('britannic', 20)
    label2 = font2.render('...for DA wannabes', 1, (0, 0, 255))
    font3 = pygame.font.SysFont('gillsansultra', 15)
    label3 = font3.render('Score: ' + str(score), 1, (0,0,255))

    surface.blit(label, (top_left_x + play_width + 8 , 30))
    surface.blit(label1, (top_left_x + play_width + 30 , 90))
    surface.blit(label2, (top_left_x + play_width + 8 , 160))
    surface.blit(label3, (top_left_x + play_width + 8, 200))
    
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            pygame.draw.rect(surface, grid[i][j], (top_left_x + j*block_size, top_left_y + i*block_size, block_size, block_size), 0)

    pygame.draw.rect(surface, (0, 0, 255), (top_left_x, top_left_y, play_width, play_height), 5)

    
def draw_text_middle(surface, text, size, color):
    font = pygame.font.SysFont('gillsansultra', size, bold=True)
    label = font.render(text, 1, color)
    surface.blit(label, (top_left_x + play_width - (label.get_width()/2), 200))
    
def main():
    global grid 
    locked_positions = {}  # This dictionary stores the coordinates' key (x,y): and color value(192,192,192) 
    grid = create_grid(locked_positions) 
    change_piece = False
    run = True
    current_piece = get_figure()
    next_piece = get_figure()
    clock = pygame.time.Clock()
    fall_time = 0    
    score = 0
        
    while run:        
        fall_speed = 0.5        
        grid = create_grid(locked_positions)
        fall_time += clock.get_rawtime()
        clock.tick()
    
        # Falling code of the piece
        if fall_time/1000 >= fall_speed:
            fall_time = 0
            current_piece.y += 1 #Calls the get_figure function and starts descending
            if not (valid_space(current_piece, grid)) and current_piece.y > 0:
                current_piece.y -= 1
                change_piece = True
                
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
         
            if event.type == pygame.KEYDOWN: #Press any key to start the game
                if event.key == pygame.K_LEFT:
                    current_piece.x -= 1
                    if not valid_space(current_piece, grid):
                        current_piece.x += 1 #The piece cannot get out of the grid
 
                elif event.key == pygame.K_RIGHT:
                    current_piece.x += 1
                    if not valid_space(current_piece, grid):
                        current_piece.x -= 1
                elif event.key == pygame.K_UP:
                    # rotate figure
                    current_piece.rotation = current_piece.rotation + 1 % len(current_piece.figure)
                    if not valid_space(current_piece, grid):
                        current_piece.rotation = current_piece.rotation - 1 % len(current_piece.figure)
 
                elif event.key == pygame.K_DOWN:
                    # move figure down
                    current_piece.y += 1
                    if not valid_space(current_piece, grid):
                        current_piece.y -= 1
                        
                if event.key == pygame.K_ESCAPE:
                    run = False
                    pygame.quit()
                    quit()
                        
        figure_pos = convert_figure_format(current_piece)
        # add color of piece to the grid for drawing
        for i in range(len(figure_pos)):
            x, y = figure_pos[i]
            if y > -1: # If we are not above the screen
                grid[y][x] = current_piece.color
       # IF PIECE HIT GROUND
        if change_piece:            
            for pos in figure_pos:
                p = (pos[0], pos[1])
                locked_positions[p] = current_piece.color
            current_piece = next_piece
            next_piece = get_figure()
            change_piece = False
            score += clear_rows(grid, locked_positions) * 100 
            clear_rows(grid, locked_positions)
            
        draw_window(window, grid, score)
        pygame.display.update()
        # Check if user lost
        if check_lost(locked_positions):
            draw_text_middle(window, "Nice try!", 40, (153,153,0))
            pygame.display.update()
            pygame.time.delay(1500)
            run = False
            
def iron_tetris():
    run = True
    while run:
        window.fill((0,0,102))
        draw_text_middle(window, 'Ready to play?', 40, (153,153,0))
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False                
            if event.type == pygame.KEYDOWN:
                main()

    pygame.display.quit()
    
window = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('IRON-TETRIS')
                           
iron_tetris()


"""
Iron Tetris was created for educational purposes. It is not intended to be sold, nor commercialized.
Part of the code was inspired by:

/***************************************************************************************
*    Title: Pygame Tetris
*    Author: Tech with Tim
*    Date: 11/25/2018
*    Code version: Version 1
*    Availability: https://techwithtim.net/tutorials/game-development-with-python/tetris-pygame/tutorial-1/
*
***************************************************************************************/

"""

'\nIron Tetris was created for educational purposes. It is not intended to be sold, nor commercialized.\nPart of the code was inspired by:\n\n/***************************************************************************************\n*    Title: Pygame Tetris\n*    Author: Tech with Tim\n*    Date: 11/25/2018\n*    Code version: Version 1\n*    Availability: https://techwithtim.net/tutorials/game-development-with-python/tetris-pygame/tutorial-1/\n*\n***************************************************************************************/\n\n'